#### Wednesday, February 12, 2025

This notebook was derived from [Agentic Rag](https://huggingface.co/docs/smolagents/examples/rag). And yeah, it had a Colab linked that went no where, so I created this from [Agentic Rag github](https://huggingface.co/docs/smolagents/examples/rag)

This all runs.

In [ ]:
# !pip install smolagents pandas langchain langchain-community sentence-transformers datasets python-dotenv rank_bm25 --upgrade -q

To call the HF Inference API, you will need a valid token as your environment variable HF_TOKEN. We use python-dotenv to load it.

In [1]:
# This expects a .env file containing the desired environment variables ... so let's create one! Be sure to add it to your .gitignore file!
from dotenv import load_dotenv
load_dotenv()

True

In [2]:
import datasets
from langchain.docstore.document import Document
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_community.retrievers import BM25Retriever

In [3]:
knowledge_base = datasets.load_dataset("m-ric/huggingface_doc", split="train")
knowledge_base = knowledge_base.filter(lambda row: row["source"].startswith("huggingface/transformers"))


README.md:   0%|          | 0.00/21.0 [00:00<?, ?B/s]

huggingface_doc.csv:   0%|          | 0.00/22.0M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/2647 [00:00<?, ? examples/s]

Filter:   0%|          | 0/2647 [00:00<?, ? examples/s]

In [4]:
source_docs = [
    Document(page_content=doc["text"], metadata={"source": doc["source"].split("/")[1]})
    for doc in knowledge_base
]

In [5]:
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=500,
    chunk_overlap=50,
    add_start_index=True,
    strip_whitespace=True,
    separators=["\n\n", "\n", ".", " ", ""],
)

In [6]:
docs_processed = text_splitter.split_documents(source_docs)

In [7]:
from smolagents import Tool

class RetrieverTool(Tool):
    name = "retriever"
    description = "Uses semantic search to retrieve the parts of transformers documentation that could be most relevant to answer your query."
    inputs = {
        "query": {
            "type": "string",
            "description": "The query to perform. This should be semantically close to your target documents. Use the affirmative form rather than a question.",
        }
    }
    output_type = "string"

    def __init__(self, docs, **kwargs):
        super().__init__(**kwargs)
        self.retriever = BM25Retriever.from_documents(
            docs, k=10
        )

    def forward(self, query: str) -> str:
        assert isinstance(query, str), "Your search query must be a string"

        docs = self.retriever.invoke(
            query,
        )
        return "\nRetrieved documents:\n" + "".join(
            [
                f"\n\n===== Document {str(i)} =====\n" + doc.page_content
                for i, doc in enumerate(docs)
            ]
        )

In [9]:
retriever_tool = RetrieverTool(docs_processed)

In [10]:
from smolagents import HfApiModel, CodeAgent

agent = CodeAgent(
    tools=[retriever_tool], model=HfApiModel(), max_steps=4, verbosity_level=2
)

In [11]:
agent_output = agent.run("For a transformers model training, which is slower, the forward or the backward pass?")

print("Final output:")
print(agent_output)

╭──────────────────────────────────────────────────── New run ────────────────────────────────────────────────────╮
│                                                                                                                 │
│ For a transformers model training, which is slower, the forward or the backward pass?                           │
│                                                                                                                 │
╰─ HfApiModel - Qwen/Qwen2.5-Coder-32B-Instruct ──────────────────────────────────────────────────────────────────╯

━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ Step 1 ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

Output message of the LLM: ────────────────────────────────────────────────────────────────────────────────────────
Thought: To determine whether the forward pass or the backward pass is slower in a transformers model training, I  
will use the `retriever` tool to find relevant information from the transformers documentation.                    
                                                                                                                   
Code:                                                                                                              
```py                                                                                                              
forward_pass_info = retriever(query="transformers model forward pass performance")                                 
backward_pass_info = retriever(query="transformers model backward pass performance")                               
print("Forward Pass Info:", forward_pass_info)                                                                     
print("Backward Pass Info:", backward_pass_info)                                                                   
```<end_code>                                                                                                      

─ Executing parsed code: ──────────────────────────────────────────────────────────────────────────────────────── 
  forward_pass_info = retriever(query="transformers model forward pass performance")                               
  backward_pass_info = retriever(query="transformers model backward pass performance")                             
  print("Forward Pass Info:", forward_pass_info)                                                                   
  print("Backward Pass Info:", backward_pass_info)                                                                 
 ─────────────────────────────────────────────────────────────────────────────────────────────────────────────────

Execution logs:
Forward Pass Info: 
Retrieved documents:


===== Document 0 =====
model_inputs = tokenizer(src_text, text_target=tgt_text, return_tensors="pt")

model(**model_inputs)  # forward pass
```

- Generation

===== Document 1 =====
>>> model = MBartForConditionalGeneration.from_pretrained("facebook/mbart-large-en-ro")
>>> # forward pass
>>> model(**inputs)
```

- Generation

  While generating the target text set the `decoder_start_token_id` to the target language id. The following
  example shows how to translate English to Romanian using the *facebook/mbart-large-en-ro* model.

```python
>>> from transformers import MBartForConditionalGeneration, MBartTokenizer

===== Document 2 =====
```python
from transformers import Tool


class HFModelDownloadsTool(Tool):
    pass
```

===== Document 3 =====
Next, regarding the debugging strategy, there are generally a few from
which to choose from:

-   Decompose the original model into many small testable components and
    run a forward pass on each of those for verification
-   Decompose the original model only into the original *tokenizer* and
    the original *model*, run a forward pass on those, and use
    intermediate print statements or breakpoints for verification

===== Document 4 =====
Next, regarding the debugging strategy, there are generally a few from which to choose from:

- Decompose the original model into many small testable components and run a forward pass on each of those for
  verification
- Decompose the original model only into the original *tokenizer* and the original *model*, run a forward pass on
  those, and use intermediate print statements or breakpoints for verification

===== Document 5 =====
layer is falsely activated during the forward pass, *i.e.* pass *self.training* to [PyTorch's functional 
dropout](https://pytorch.org/docs/stable/nn.functional.html?highlight=dropout#torch.nn.functional.dropout)

===== Document 6 =====
- For languages with a Roman alphabet, such as English or French, the tokenizer can be used directly to 
pre-process the text inputs. The following code example runs a forward pass using the MMS-TTS English checkpoint:

```python
import torch
from transformers import VitsTokenizer, VitsModel, set_seed

tokenizer = VitsTokenizer.from_pretrained("facebook/mms-tts-eng")
model = VitsModel.from_pretrained("facebook/mms-tts-eng")

===== Document 7 =====
Having managed to correctly load the pretrained weights into the 🤗 Transformers implementation, you should now 
make
sure that the forward pass is correctly implemented. In [Get familiar with the original 
repository](#34-run-a-pretrained-checkpoint-using-the-original-repository), you have already created a script that 
runs a forward
pass of the model using the original repository. Now you should write an analogous script using the 🤗 Transformers

===== Document 8 =====
4.  [ ] Created script that successfully runs forward pass using
    original repository and checkpoint

5.  [ ] Successfully opened a PR and added the model skeleton to Transformers

6.  [ ] Successfully converted original checkpoint to Transformers
    checkpoint

7.  [ ] Successfully ran forward pass in Transformers that gives
    identical output to original checkpoint

8.  [ ] Finished model tests in Transformers

9.  [ ] Successfully added Tokenizer in Transformers

===== Document 9 =====
4.  [ ] Created script that successfully runs forward pass using
    original repository and checkpoint

5.  [ ] Successfully opened a PR and added the model skeleton to Transformers

6.  [ ] Successfully converted original checkpoint to Transformers
    checkpoint

7.  [ ] Successfully ran forward pass in Transformers that gives
    identical output to original checkpoint

8.  [ ] Finished model tests in Transformers

9.  [ ] Successfully added Tokenizer in Transformers
Backward Pass Info: 
Retrieved documents:


===== Document 0 =====
Saving all activations from the forward pass in order to compute the gradients during th

...

Output message of the LLM: ────────────────────────────────────────────────────────────────────────────────────────
Thought: Based on the information retrieved, the backward pass generally involves more computations than the       
forward pass, especially for layers like convolutions and linear layers, where there are approximately 2x flops in 
the backward pass compared to the forward pass. This typically results in the backward pass being slower than the  
forward pass. I will now formulate the final answer based on this information.                                     
                                                                                                                   
Code:                                                                                                              
```py                                                                                                              
final_answer("backward")                                                                                           
```<end_code>                                                                                                      

─ Executing parsed code: ──────────────────────────────────────────────────────────────────────────────────────── 
  final_answer("backward")                                                                                         
 ─────────────────────────────────────────────────────────────────────────────────────────────────────────────────

Out - Final answer: backward

[Step 1: Duration 6.22 seconds| Input tokens: 6,028 | Output tokens: 190]

Final output:
backward
